# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [31]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [32]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

#df_densidad_alta = pd.read_csv("instancias/densidad-alta/indice.csv");
#df_densidad_baja = pd.read_csv("instancias/densidad-baja/indice.csv");
#df_mejor_caso_bt = pd.read_csv("instancias/mejor-caso-bt/indice.csv");
#df_peor_caso_bt = pd.read_csv("instancias/peor-caso-bt/indice.csv");
#df_dinamica = pd.read_csv("instancias/dinamica/indice.csv");
bt_podas = pd.read_csv("instancias/podas-BT/indice.csv");
bt_vs_pd = pd.read_csv("instancias/BTvsPD/indice.csv");

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.
- FB: Fuerza bruta
- BT: Backtracking con ambas podas.
- BT-F: Backtracking solamente con poda por factibilidad.
- BT-O: Backtracking solamente con poda por optimalidad.
- DP: Programación dinámica.

In [33]:
# Devuelve el tiempo de ejecucion
def correr_experimento(metodo, archivo_instancia): #archivo_instancia es el PATH al test.
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../social_distancing", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar. OKEY. ACA. 
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return tiempo_de_ejecucion;

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [34]:
experimentos = [];

### Experimento 1
Correr Fuerza Bruta en las primeras 30 instancias de densidad-alta y densidad-baja.

Mio: Correr FB de la instancia 30 a la 60 de ambas densidades.

In [11]:
for n in range(10, 35):
    fila_n = df_densidad_alta.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "FB", fila_n["archivo"]]);

for n in range(10, 35):
    fila_n = df_densidad_baja.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "FB", fila_n["archivo"]]);

## Experimento 2
Correr BT para mejor y peor caso.

In [23]:
for n in range(0, df_mejor_caso_bt.shape[0]):
    fila_n = df_mejor_caso_bt.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT", fila_n["archivo"]]);

#for n in range(0, df_peor_caso_bt.shape[0]):
 #   fila_n = df_peor_caso_bt.iloc[n];
  #  experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT", fila_n["archivo"]]);

## Experimento 3
Correr BT, BT-F, BT-O para densidad alta y baja.

In [ ]:
for n in range(0, df_densidad_alta.shape[0]):
    fila_n = df_densidad_alta.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT-F", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT-O", fila_n["archivo"]]);

for n in range(0, 30):
    fila_n = df_densidad_baja.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT-F", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT-O", fila_n["archivo"]]);

# Experimento 4
Correr DP para el dataset _dinamica_.

In [22]:
for i in range(0, df_dinamica.shape[0]):
    fila = df_dinamica.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["W"], "DP", fila["archivo"]]);

## Experimento 5
Correr DP para el dataset densidad-alta y densidad-baja.

In [7]:
for i in range(0, df_densidad_alta.shape[0]):
    fila = df_densidad_alta.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["W"], "DP", fila["archivo"]]);

#for i in range(0, df_densidad_baja.shape[0]):
  #  fila = df_densidad_baja.iloc[i];
   # experimentos.append([fila["dataset"], fila["n"], fila["W"], "DP", fila["archivo"]]);

NameError: name 'df_densidad_alta' is not defined

## Experimento 6
Correr BT para las combinaciones de densidad alta/baja con beneficios iguales o dispares.

```Riesgo Alto - Igual Beneficio ```

In [8]:
for i in range(0, bt_podas.shape[0]):
    fila = bt_podas.iloc[i];
    if (fila["instancia"].find("RiesgoAlto-IgualBeneficios") >= 0):
        experimentos.append([fila["dataset"] + "-RA-BI", fila["n"], fila["W"], "BT", fila["archivo"]]);
        experimentos.append([fila["dataset"] + "-RA-BI", fila["n"], fila["W"], "BT-O", fila["archivo"]]);
        experimentos.append([fila["dataset"] + "-RA-BI", fila["n"], fila["W"], "BT-F", fila["archivo"]]);

```Riesgo Alto - Beneficios Desiguales ```

In [11]:
for i in range(0, bt_podas.shape[0]):
    fila = bt_podas.iloc[i];
    if (fila["instancia"].find("RiesgoAlto-BeneficiosDesiguales") >= 0):
        experimentos.append([fila["dataset"] + "-RA-BD", fila["n"], fila["W"], "BT", fila["archivo"]]);
        experimentos.append([fila["dataset"] + "-RA-BD", fila["n"], fila["W"], "BT-O", fila["archivo"]]);
        experimentos.append([fila["dataset"] + "-RA-BD", fila["n"], fila["W"], "BT-F", fila["archivo"]]);

```Riesgo Bajo - Igual Beneficio ```

In [14]:
for i in range(0, bt_podas.shape[0]):
    fila = bt_podas.iloc[i];
    if (fila["instancia"].find("RiesgoBajo-IgualBeneficios") >= 0):
        experimentos.append([fila["dataset"] + "-RB-BI", fila["n"], fila["W"], "BT", fila["archivo"]]);
        experimentos.append([fila["dataset"] + "-RB-BI", fila["n"], fila["W"], "BT-O", fila["archivo"]]);
        experimentos.append([fila["dataset"] + "-RB-BI", fila["n"], fila["W"], "BT-F", fila["archivo"]]);

```Riesgo Bajo - Beneficios Desiguales ```

In [17]:
for i in range(0, bt_podas.shape[0]):
    fila = bt_podas.iloc[i];
    if (fila["instancia"].find("RiesgoBajo-BeneficiosDesiguales") >= 0):
        experimentos.append([fila["dataset"] + "-RB-BD", fila["n"], fila["W"], "BT", fila["archivo"]]);
        experimentos.append([fila["dataset"] + "-RB-BD", fila["n"], fila["W"], "BT-O", fila["archivo"]]);
        experimentos.append([fila["dataset"] + "-RB-BD", fila["n"], fila["W"], "BT-F", fila["archivo"]]);

## Experimento 7
Correr PD y BT en dataset de densidad alta y baja para ver como se comportan dichos algoritmos

```Densidad de riesgo baja ```

In [35]:
for i in range(0, bt_vs_pd.shape[0]):
    fila = bt_vs_pd.iloc[i];
    if (fila["instancia"].find("DensidadBaja") >= 0):
        experimentos.append([fila["dataset"] + "-DRB", fila["n"], fila["W"], "BT", fila["archivo"]]);
        experimentos.append([fila["dataset"] + "-DRB", fila["n"], fila["W"], "DP", fila["archivo"]]);
        experimentos.append([fila["dataset"] + "-DRB", fila["n"], fila["W"], "BT-F", fila["archivo"]]);

```Densidad de riesgo alta ```

In [35]:
for i in range(0, bt_vs_pd.shape[0]):
    fila = bt_vs_pd.iloc[i];
    if (fila["instancia"].find("DensidadAlta") >= 0):
        experimentos.append([fila["dataset"] + "-DRA", fila["n"], fila["W"], "BT", fila["archivo"]]);
        experimentos.append([fila["dataset"] + "-DRA", fila["n"], fila["W"], "DP", fila["archivo"]]);
        experimentos.append([fila["dataset"] + "-DRA", fila["n"], fila["W"], "BT-F", fila["archivo"]]);

## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [36]:
columnas = ["dataset", "n", "W", "metodo", "tiempo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
#print(experimentos)
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    for i in range(0, T):
        #print(experimento[3])
        tiempos.append(correr_experimento(experimento[3], experimento[4]));
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo]);
    print(filas)
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado.csv", index=False, header=True);

'Experimento: 120/120'

[['BTvsPD-DRB', 1, 1, 'BT', 0.002444], ['BTvsPD-DRB', 1, 1, 'DP', 0.002765], ['BTvsPD-DRB', 1, 1, 'BT-F', 0.002344], ['BTvsPD-DRB', 2, 1, 'BT', 0.002314], ['BTvsPD-DRB', 2, 1, 'DP', 0.002695], ['BTvsPD-DRB', 2, 1, 'BT-F', 0.002274], ['BTvsPD-DRB', 3, 1, 'BT', 0.002855], ['BTvsPD-DRB', 3, 1, 'DP', 0.003607], ['BTvsPD-DRB', 3, 1, 'BT-F', 0.002294], ['BTvsPD-DRB', 4, 2, 'BT', 0.002795], ['BTvsPD-DRB', 4, 2, 'DP', 0.003066], ['BTvsPD-DRB', 4, 2, 'BT-F', 0.002505], ['BTvsPD-DRB', 5, 2, 'BT', 0.003386], ['BTvsPD-DRB', 5, 2, 'DP', 0.003927], ['BTvsPD-DRB', 5, 2, 'BT-F', 0.002625], ['BTvsPD-DRB', 6, 3, 'BT', 0.002485], ['BTvsPD-DRB', 6, 3, 'DP', 0.004028], ['BTvsPD-DRB', 6, 3, 'BT-F', 0.002494], ['BTvsPD-DRB', 7, 3, 'BT', 0.003066], ['BTvsPD-DRB', 7, 3, 'DP', 0.004559], ['BTvsPD-DRB', 7, 3, 'BT-F', 0.003216], ['BTvsPD-DRB', 8, 4, 'BT', 0.003907], ['BTvsPD-DRB', 8, 4, 'DP', 0.004338], ['BTvsPD-DRB', 8, 4, 'BT-F', 0.003607], ['BTvsPD-DRB', 9, 4, 'BT', 0.003096], ['BTvsPD-DRB', 9, 4, 'DP', 0.0049